# LSTM-VAE Training Notebook

## Load libraries and setup

In [48]:

import torch
import torch.nn as nn
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error



In [49]:
class LSTMVAE(nn.Module):
    def __init__(self, num_input, num_hidden, num_layers, dropout, output_window, output_dim):
        super(LSTMVAE, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = num_hidden
        self.output_window = output_window
        self.output_dim = output_dim

        # Encoder
        self.encoder = nn.LSTM(num_input, num_hidden, num_layers, batch_first=True, dropout=dropout)

        # Latent space
        self.scale = nn.Sequential(nn.Linear(num_hidden, num_hidden), nn.Tanh())
        self.mu = nn.Linear(num_hidden, num_hidden)
        self.log_var = nn.Linear(num_hidden, num_hidden)

        # Decoder: Linear layer maps from latent to full output window of target variable
        self.decoder = nn.Linear(num_hidden, output_window)

    def reparametrize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x):
        _, (h_n, _) = self.encoder(x)  # shape: (num_layers, batch, hidden_size)
        return self.scale(h_n[-1])     # shape: (batch, hidden_size)

    def decode(self, encoded, z):
        out = encoded * (1 + z)  # shape: (batch, hidden_size)
        # out = encoded #* (1 + z)  # shape: (batch, hidden_size)
        decoded = self.decoder(out)  # shape: (batch, output_window * output_dim)
        return decoded.view(-1, self.output_window)#, self.output_dim)  # reshape to (batch, 100, 1)

    def forward(self, x):
        encoded = self.encode(x)
        mu = self.mu(encoded)
        log_var = self.log_var(encoded)
        z = self.reparametrize(mu, log_var)
        decoded = self.decode(encoded, z)
        return decoded, mu, log_var

    def sample(self, x, num_samples):
        encoded = self.encode(x).unsqueeze(1).repeat(1, num_samples, 1)
        z = torch.randn((x.size(0), num_samples, self.hidden_size)).to(x.device)
        decoded = self.decoder(encoded * (1 + z))
        return decoded.view(x.size(0), num_samples, self.output_window, self.output_dim)


In [50]:
def loss(x_hat, x_target, mu, log_var):
    recon_loss = nn.functional.mse_loss(x_hat.squeeze(-1), x_target.squeeze(-1), reduction='mean')
    weight = (x_target.squeeze(-1) != 0).float() + 1
    weighted_recon_loss = torch.mean(weight * recon_loss)
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) / x_hat.size(0)
    return weighted_recon_loss, kl_loss

In [51]:
def create_sequences(dataframe, input_window=500, output_window=1, step=24, offset=100):
    appliances_idx = dataframe.columns.get_loc("Appliances")

    X_data = dataframe.values  # full multivariate data
    y_data = dataframe["Appliances"].values  # target column

    X, y = [], []
    for i in range(0, len(dataframe) - input_window - offset - output_window + 1, step):
        X.append(X_data[i : i + input_window])
        y.append(y_data[i + input_window + offset : i + input_window + offset + output_window])

    X = np.array(X)
    y = np.array(y)

    print(f"✅ X shape: {X.shape} (samples, input_window, input_features)")
    print(f"✅ y shape: {y.shape} (samples, output_window)")

    return X, y


def safe_rolling_sum(df, column="Appliances", window=7*24*6):  # 1008
    values = df[column].values.astype(np.float64)
    cum = np.cumsum(np.insert(values, 0, 0))  # Pad with zero for correct diff
    result = cum[window:] - cum[:-window]
    padded_result = np.concatenate([np.full(window-1, result[0]), result])
    df['Appliances_cumulative'] = padded_result
    return df


def get_dataloaders(csv_path, input_window=500, output_window=100, offset=100):
    df = pd.read_csv(csv_path, index_col=0, parse_dates=True)

    # OPTIONAL: You can print this to confirm column names
    # print(df.columns)
    batch_size = 32
    # Ensure all values are float32 except the index
    df = df.astype(np.float32)

    df['Appliances'] = df['Appliances'].rolling(6*6, min_periods=1).mean()
    # df['Appliances'] = np.log1p(df['Appliances'])

    # Create sequences using the DataFrame (so we can access column names)
    X, y = create_sequences(df, input_window=input_window, output_window=output_window, offset=offset)

    # Convert to PyTorch tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    # Split (e.g., 70% train, 15% val, 15% test)
    total_samples = len(X)
    train_end = int(0.7 * total_samples)
    val_end = int(0.85 * total_samples)

    train_dataset = torch.utils.data.TensorDataset(X[:train_end], y[:train_end])
    val_dataset = torch.utils.data.TensorDataset(X[train_end:val_end], y[train_end:val_end])
    test_dataset = torch.utils.data.TensorDataset(X[val_end:], y[val_end:])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

## Load data

In [52]:
# Ensure the path is absolute and points to the same directory as this notebook
data_path = os.path.join(os.getcwd(), "processed_energy.csv")

# Now load
train_loader, val_loader, _ = get_dataloaders(csv_path=data_path, input_window=100, output_window=1)


✅ X shape: (814, 100, 26) (samples, input_window, input_features)
✅ y shape: (814, 1) (samples, output_window)


## Initialize model

In [53]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMVAE(num_input=26, num_hidden=128, num_layers=2, dropout=0.3, output_window=1, output_dim=1)
model.to(device)


LSTMVAE(
  (encoder): LSTM(26, 128, num_layers=2, batch_first=True, dropout=0.3)
  (scale): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): Tanh()
  )
  (mu): Linear(in_features=128, out_features=128, bias=True)
  (log_var): Linear(in_features=128, out_features=128, bias=True)
  (decoder): Linear(in_features=128, out_features=1, bias=True)
)

## Define loss and optimizer

In [54]:

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
reconstruction_loss_fn = torch.nn.MSELoss()


## Training loop

In [55]:
def evaluate(model, loader, device):
    model.eval()
    recon_total, kl_total = 0.0, 0.0
    preds, targets = [], []

    with torch.no_grad():
        for x_batch, y_batch in loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_hat, mu, log_var = model(x_batch)
            recon_loss, kl_loss = loss(x_hat, y_batch, mu, log_var)

            recon_total += recon_loss.item() * x_batch.size(0)
            kl_total += kl_loss.item() * x_batch.size(0)

            preds.append(x_hat.cpu())
            targets.append(y_batch.cpu())

    preds = torch.cat(preds, dim=0).squeeze(-1).numpy()
    targets = torch.cat(targets, dim=0).squeeze(-1).numpy()

    mae = mean_absolute_error(targets, preds)
    rmse = mean_squared_error(targets, preds, squared=False)

    return recon_total / len(loader.dataset), kl_total / len(loader.dataset), mae, rmse


In [56]:
def train_lstmvae(model, train_loader, val_loader, device, epochs=10, save_path=None):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    
    for epoch in range(epochs):
        model.train()
        recon_total, kl_total = 0.0, 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            x_hat, mu, log_var = model(x)
            recon, kl = loss(x_hat, y, mu, log_var)
            (recon + 1e-2 * kl).backward()
            optimizer.step()
            recon_total += recon.item() * x.size(0)
            kl_total += kl.item() * x.size(0)
        
        val_recon, val_kl, val_mae, val_rmse = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | TrainRecon: {recon_total/len(train_loader.dataset):.4f} "
              f"| KL: {kl_total/len(train_loader.dataset):.4f} | ValRecon: {val_recon:.4f} "
              f"| KL: {val_kl:.4f} | MAE: {val_mae:.4f} | RMSE: {val_rmse:.4f}")
    
    if save_path:
        torch.save(model.state_dict(), "lstmvae_1step.pth")
        print(f"✅ Model saved to {save_path}")



In [57]:
# Paths
data_path = os.path.join("processed_energy.csv")
model_save_path = os.path.join("lstmvae_1step.pth")  # You can still nest outputs

# Initialize and train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader, _ = get_dataloaders(data_path, 100, 1)
model = LSTMVAE(26, 128, 2, 0.3, 1, 1)
train_lstmvae(model, train_loader, val_loader, device, epochs=10, save_path=model_save_path)


✅ X shape: (814, 100, 26) (samples, input_window, input_features)
✅ y shape: (814, 1) (samples, output_window)
Epoch 1/10 | TrainRecon: 0.0102 | KL: 0.1193 | ValRecon: 0.0052 | KL: 0.0130 | MAE: 0.0400 | RMSE: 0.0509
Epoch 2/10 | TrainRecon: 0.0079 | KL: 0.0070 | ValRecon: 0.0049 | KL: 0.0039 | MAE: 0.0392 | RMSE: 0.0494
Epoch 3/10 | TrainRecon: 0.0072 | KL: 0.0036 | ValRecon: 0.0043 | KL: 0.0024 | MAE: 0.0347 | RMSE: 0.0461
Epoch 4/10 | TrainRecon: 0.0067 | KL: 0.0021 | ValRecon: 0.0047 | KL: 0.0016 | MAE: 0.0337 | RMSE: 0.0484
Epoch 5/10 | TrainRecon: 0.0069 | KL: 0.0014 | ValRecon: 0.0046 | KL: 0.0012 | MAE: 0.0312 | RMSE: 0.0479
Epoch 6/10 | TrainRecon: 0.0067 | KL: 0.0009 | ValRecon: 0.0046 | KL: 0.0008 | MAE: 0.0377 | RMSE: 0.0482
Epoch 7/10 | TrainRecon: 0.0066 | KL: 0.0009 | ValRecon: 0.0043 | KL: 0.0007 | MAE: 0.0322 | RMSE: 0.0466
Epoch 8/10 | TrainRecon: 0.0063 | KL: 0.0006 | ValRecon: 0.0043 | KL: 0.0007 | MAE: 0.0317 | RMSE: 0.0465
Epoch 9/10 | TrainRecon: 0.0062 | KL: 0.0

## Save model